In [3]:
import cv2
import numpy as np
import os

def find_inter_means_threshold(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Initialize the threshold as the mean of the grayscale image
    threshold = np.mean(gray_image)

    while True:
        # Calculate the means of two regions: pixels below and above the threshold
        region1 = gray_image[gray_image <= threshold]
        region2 = gray_image[gray_image > threshold]

        mean1 = np.mean(region1) if len(region1) > 0 else 0
        mean2 = np.mean(region2) if len(region2) > 0 else 0

        # Calculate the new threshold as the average of the two means
        new_threshold = (mean1 + mean2) / 2

        # If the new threshold is very close to the previous one, break the loop
        if abs(threshold - new_threshold) < 1e-2:
            break

        threshold = new_threshold

    return int(threshold)

def segment_image(image, threshold):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Create a binary mask based on the threshold
    segmented_image = (gray_image > threshold).astype(np.uint8) * 255

    return segmented_image

# Function to find the threshold and segment the image
def process_image(image, output_folder):
    threshold = find_inter_means_threshold(image)
    segmented_image = segment_image(image, threshold)
    return segmented_image

# Load your images from the specified folder
input_folder = './data/'  # Replace with the actual folder path
output_folder = './results/'  # Replace with the desired output folder

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  # You can add more image file extensions
        image = cv2.imread(os.path.join(input_folder, filename))
        if image is not None:
            segmented_image = process_image(image, output_folder)
            output_filename = os.path.splitext(filename)[0] + '_segmented.jpg'
            output_path = os.path.join(output_folder, output_filename)
            cv2.imwrite(output_path, segmented_image)

# Display or further process segmented images as needed
